In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('../data/train.csv', index_col=0)

In [3]:
print(df.columns)
print(df.shape)

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked'],
      dtype='object')
(891, 11)


In [4]:
df.sample(5)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
238,1,2,"Collyer, Miss. Marjorie ""Lottie""",female,8.0,0,2,C.A. 31921,26.250,NaN,S
417,1,2,"Drew, Mrs. James Vivian (Lulu Thorne Christian)",female,34.0,1,1,28220,32.500,NaN,S
655,0,3,"Hegarty, Miss. Hanora ""Nora""",female,18.0,0,0,365226,6.750,NaN,Q
401,1,3,"Niskanen, Mr. Juha",male,39.0,0,0,STON/O 2. 3101289,7.925,NaN,S
652,1,2,"Doling, Miss. Elsie",female,18.0,0,1,231919,23.000,NaN,S


In [5]:
# The age column has nas, but they don't seem special at first sight.
# I'll fill them with the median

df[df['Age'].isnull()].sample(5)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
829,1,3,"McCormack, Mr. Thomas Joseph",male,NaN,0,0,367228,7.7500,NaN,Q
565,0,3,"Meanwell, Miss. (Marion Ogden)",female,NaN,0,0,SOTON/O.Q. 392087,8.0500,NaN,S
476,0,1,"Clifford, Mr. George Quincy",male,NaN,0,0,110465,52.0000,A14,S
336,0,3,"Denkoff, Mr. Mitto",male,NaN,0,0,349225,7.8958,NaN,S
564,0,3,"Simmons, Mr. John",male,NaN,0,0,SOTON/OQ 392082,8.0500,NaN,S


In [6]:
# Substitute missing ages with the median (28)

age = df['Age'][:]
median_age = age.median()
age[age.isnull()] = median_age

df2 = df[:]
df2['Age'] = age

In [7]:
# Encode cabin as 'data present' vs 'absent'
df2['Cabin'] = ~df2['Cabin'].isnull()

In [8]:
discrete_columns = ['Pclass','Sex', 'Cabin', 'Embarked']
dummies = []

# Workaround to get pandas to behave as I want
# Otherwise it keeps multilevel integer variables in a single column
for column in discrete_columns:
    
    these_dummies = pd.get_dummies(df2[column], prefix=column, drop_first=True)
    dummies.append(these_dummies)
    
discrete = pd.concat(dummies, axis=1)

In [9]:
continuous_columns = ['Age', 'Fare']

continuous = df2[continuous_columns]

continuous_normalized = (continuous - continuous.mean()) / (continuous.max() - continuous.min())

In [10]:
features = pd.concat([discrete, continuous_normalized], axis=1)

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(features, df2['Survived'], test_size=0.3)

print(X_train.shape)
print(X_test.shape)

(623, 8)
(268, 8)


In [17]:
from sklearn.ensemble.forest import RandomForestClassifier

forest = RandomForestClassifier()
model = forest.fit(X_train, Y_train)
predicted_probs = model.predict_proba(X_test)
predictions = model.predict(X_test)

In [18]:
from sklearn.metrics import roc_curve, roc_auc_score

roc = roc_curve(Y_test, predicted_probs[:,1])
roc_auc_score(validation['Survived'], predicted_probs[:,1])

NameError: name 'validation' is not defined

In [ ]:
plt.plot(roc[0],roc[1])

Decent enough! Let's play around a bit more

In [ ]:
# SVMs now

